# ADVANCED STATISTICS FOR PHYSICS ANALYSIS, University of Padova

__Authors:__

- Clara Eminente (University of Padova)
- Matteo Guida (University of Padova)

__Supervised by:__

- Professor Alberto Garfagnini (University of Padova, INFN)

# Introduction

In the following a spectrum coming from the combination of three sources and collected by a Germanium detector is investigated.
</p>
We recognize in the spectrum the following processes which are going to be analyzed. 

<img>
<img src="IMMAGINE_COMMENTATA.png",width=9000,height=9000>

$\begin{aligned}
& \text{AMERICIUM}  \\
& ^{241\!\,}_{\ 95}Am\ \overset{432.2y}{\longrightarrow} \ ^{237}_{\ 93}Np~+~^{4}_{2}\alpha^{2+} +\gamma \,\,\,\, E_{\gamma}=0.059 \,\, \text{MeV} \\
\\
\\
& \text{COBALT}  \\
& ^{60\!\,}_{\ 27}Co\ \overset{5.27y}{\longrightarrow} \ ^{60}_{\ 28}Ni~+~e^{-} + \bar{\nu}_{e} \\
\\
& ^{60}_{\ 28}Ni {\longrightarrow} \ ^{60}_{\ 28}Ni + \gamma  \,\,\,\,\,\,\,\,\,\, E_{\gamma}=1.17\,\, \text{MeV} \\
& ^{60}_{\ 28}Ni {\longrightarrow} \ ^{60}_{\ 28}Ni + \gamma   \,\,\,\,\,\,\,\,\,\, E_{\gamma}=1.33\,\, \text{MeV}
\\
\\
& \text{CESIUM}  \\
& ^{137\!\,}_{\ 55}Cs\ \overset{ 30.17y}{\longrightarrow} \ ^{137}_{\ 56}Ba~+~e^{-}+ \bar{\nu}_{e} \\
& ^{137}_{\ 56}Ba {\longrightarrow} \ ^{137}_{\ 56}Ba + \gamma   \,\,\,\,\,\,\,\,\,\, E_{\gamma}=0.66\,\, \text{MeV}
\end{aligned}$

# Goal
The existimation of the number of counts under the peaks due to the previous processes. 

# General analysis strategy

We do not have a unique model to describe all the processes that are present in the spectrum at once.
</p>
</p>
For all the processes analyzed we perform the following operations:
    
1. Consider a reduced range of channels
2. Choose a peak shape function
3. Choose a "background" function

For the peak shape the function from which we start is Gaussian due to the energy resolution in the detector. We can make the model more complex studying the additional tail. 

The background in the region of the peak has 2 main components:
1. Pulses related to radiation from other sources.
2. Pulses from the desired $\gamma$-rays, whose energy escapes from the sensitive volume of the detector.

The best modeling function for the background is evaluated in each individual case.


## Computational tools: JAGS

In order to obtain a distribution for the number of counts we implemented a Gibbs sampler using the package rJAGS from which we extracted the Markov Chain Monte Carlo (MCMC) of the parameters of the chosen generative model.

The generative model is evaluated in each individual case according to the shape of the peak and the background.

The chosen noise (or measurement) model is a Poisson distribution as we know that once the energy (channel) is fixed the distribution of counts follows this kind of distribution with a mean value given by the value of generative model in that point. Thus:
\begin{equation*}
p(n) = \frac{\lambda^{n}e^{-\lambda}}{n!}
\end{equation*}
- $\lambda$ = value from generative model 
- n = measured value


# SET UP

## Libraries

In [ ]:
library(ggplot2) 
library(rjags)
library(repr)
library(fBasics)
library(latex2exp)

## Data Loading

In [ ]:
counts<-read.delim("B19036_AmCsCo_20180316.dat", skip =2,header = TRUE)
# ln.counts=log(as.numeric(counts[,1]))
# par(mfrow=c(1,1))
# plot(ln.counts[ln.counts>0], type = 'l', col='red', main = 'Am−Cs−Co spectra', xlab = 'ADC channel', ylab = "log(counts)",cex.main = 1.5)

## Calibration Curve

In [ ]:
ADC_channel <-seq(1, NROW(na.omit(counts)), by=1)
dataset <- data.frame(counts,ADC_channel)
colnames(dataset) <- c("Counts","ADC_channel")
dataset<-dataset[!(dataset$Counts<=0),]
plot(dataset$ADC_channel,dataset$Counts, type = 'l', col='red', main = 'Am−Cs−Co spectra', xlab = 'ADC channel [#]', ylab = "Counts [#]",cex.main = 1.5)

In [ ]:
range1 = dataset[!(dataset$ADC_channel<=70) & (dataset$ADC_channel<=200),]
max1=range1[which.max(range1$Counts),"ADC_channel"]
# cat(paste("max1 : ",which.max(range1$Counts),"\n"))
range2 = dataset[!(dataset$ADC_channel<=1500) & (dataset$ADC_channel<=2000),]
max2=range2[which.max(range2$Counts),"ADC_channel"]
# cat(paste("max2 : ",max2,"\n"))
range3 = dataset[!(dataset$ADC_channel<=3000) & (dataset$ADC_channel<=3100),]
max3=range3[which.max(range3$Counts),"ADC_channel"]
# cat(paste("max3 : ",max3,"\n"))
range4 = dataset[!(dataset$ADC_channel<=3200) & (dataset$ADC_channel<=3700),]
max4=range4[which.max(range4$Counts),"ADC_channel"]
# cat(paste("max4 : ",max4,"\n"))
range5 = dataset[!(dataset$ADC_channel<=6000) & (dataset$ADC_channel<=7000),]
max5=range5[which.max(range5$Counts),"ADC_channel"]
# cat(paste("max5 : ",max5,"\n"))

In [ ]:
a1<-c(158,1723,3052,3466,6504) #MEANS FOUND AFTERWARDS
x<-c(max1,max2,max3,max4,max5) #THEORETHICAL MAX.ENERGY (IN CHANNEL)
# error.x
for(i in 1:length(x))
{
x1<-x-a1[i]
y<-c(0.0595,0.6617,1.1732,1.3325,2.5057)
data <- data.frame(x1,y)
calibration_curve2 <- lm(y ~ x1, data=data) 
summary(calibration_curve2)
ggplot(data, aes(x1, y)) +
  geom_point(aes()) +
  geom_smooth(method = "lm") + labs(y = "Energy [MeV]", x = "ADC channel")+ ggtitle(" Calibration Curve")+
  theme(plot.title = element_text(size = 20, face = "bold",hjust = 0.5))+theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
panel.background = element_blank(), axis.line = element_line(colour = "black"))
energy.1 <- summary(calibration_curve2)$coefficients[1,1]
error.energy.1 <- summary(calibration_curve2)$coefficients[1,2]
cat(paste(energy.1*10**3, " +-",error.energy.1*10**3,"\n" ))      

}

# SUMMARY TABLE

In [ ]:
name <- c("Americium (Am)","Cesium (Cs)","Cobaltum1 (Co1)","Cobaltum2 (Co2)", "Double Absorption (DA)")
argmax.channel <- c(max1,max2,max3,max4,max5)
energy.MeV <- c(0.059,0.66,1.17,1.33,2.506)
range.gauss.min <- c(120,1650,2950,3300,6000)
range.gauss.max <- c(180,1790,3170,3600,6700)


In [ ]:
summary.table <- data.frame(name,argmax.channel,energy.MeV,range.gauss.min,range.gauss.max)
row.names(summary.table) <- c("Am","Cs","Co1","Co2","DA")
summary.table

# FUNCTIONS

## Generative Model Functions 

In [ ]:
Gaussian.func <-function(x,mu,sigma,N){
    return((N/sqrt(2*pi*sigma*sigma)*exp((-(x-mu)*(x-mu))/(2*sigma*sigma))))
}

In [ ]:
Gauss.Sig.func <- function(x, c.1, c.2, c.mu, N, mu, sigma, offset){
    return( (c.1/(1+exp(c.2*(x-c.mu)))) + (N/sqrt(2*pi*sigma*sigma)*exp((-(x-mu)*(x-mu))/(2*sigma*sigma))) + offset)
}

In [ ]:
Routti.Prussin.peak<-function(z,J,A,mu,sigma)
    {
    ifelse(z < mu - J,  A/(sigma)*exp(J*(2*z - 2*mu + J)/(2*sigma*sigma)),
           N/(sqrt(2*pi)*sigma)*exp((-(z-mu)*(z-mu))/(2*sigma*sigma)))
    }

In [ ]:
Gauss.line.func <- function(x,slope, offset, N, mu, sigma){
    return( (N/sqrt(2*pi*sigma*sigma)*exp((-(x-mu)*(x-mu))/(2*sigma*sigma))) + (x-mu)*tan(slope) + offset)
}

In [ ]:
Gauss.exp.func <- function(x,slope, offset, N, mu, sigma){
    return( (N/sqrt(2*pi*sigma*sigma)*exp((-(x-mu)*(x-mu))/(2*sigma*sigma))) + exp((x-mu)*tan(slope) + offset))
}

In [ ]:
piecewise<-function(z,J,A,c.1,c.2,c.mu,mu,offset,sigma)
    {
    ifelse(z < mu - J,(c.1/(1 + exp(c.2*(z - c.mu))) + offset +  A/(sigma)*exp(J*(2*z - 2*mu + J)/(2*sigma*sigma))),
           (c.1/(1 + exp(c.2*(z - c.mu))) + offset + A/(sigma)*exp((-(z-mu)*(z-mu))/(2*sigma*sigma))))
    }


## Plotting functions

In [ ]:
plot.func <- function(x.points, y.points, extracted, sigma, title){
    
    options(repr.plot.width=10, repr.plot.height=7)
    plot(x.points, extracted,col="navy", main=title)
    points(x.points, y.points, col="red")
    #arrows(x.points, extracted-sigma, x.points, extracted+sigma, length=0.05, angle=90, code=3, col="navy")
    legend("topright",legend=c("Generated","Data"),col=c("navy","red"),pch=15)
   
}

In [ ]:
plot.func.total <- function(x.points, y.points, extracted, extracted.mode, sigma, sigma.mode, title){
    
    options(repr.plot.width=10, repr.plot.height=7)
    plot(x.points, extracted,col="navy", main=title)
    points(x.points, y.points, col="red")
    arrows(x.points, extracted-sigma, x.points, extracted+sigma, length=0.05, angle=90, code=3, col="navy")
    
    points(x.points, extracted.mode, col="darkgoldenrod1")
    arrows(x.points, extracted.mode-sigma.mode, x.points, extracted.mode+sigma.mode, length=0.05, angle=90, code=3, col="darkgoldenrod1")
    
    
    legend("topright",legend=c("Generated","Data","Generated\nmode"),col=c("navy","red","darkgoldenrod1"),pch=15)
   
}

In [ ]:
plot.func.total.1 <- function(x.points, y.points, extracted, extracted.mode, extracted.mean, sigma, title){
    
    options(repr.plot.width=10, repr.plot.height=7)
    
    y.lim=max(c(max(y.points)+sqrt(max(extracted)),max(extracted.mode),max(extracted),max(extracted.mean)))
    
    plot(x.points, y.points, col="navy", main=title, ylim=c(0,y.lim),pch=0.1,xlab="Channel[#]",ylab="Counts[#]")
    arrows(x.points, y.points-sqrt(extracted), x.points, y.points+sqrt(extracted), length=0.05, angle=90, code=3, col="navy")
        
    points(x.points, extracted, col="red", type="l",lwd=3, lty=4)
    points(x.points, extracted.mode, col="darkgoldenrod1", type="l",lwd=3, lty=2)
    points(x.points, extracted.mean, col="darkolivegreen4", type="l",lwd=3, lty=3)
    
    
    
    legend("topleft",legend=c(TeX("$Data[i]  \\, ±  \\,\\sqrt{generated[i]}$"),"Generated","Generated mode", "Generated mean"),col=c("navy","red","darkgoldenrod1","darkolivegreen4"),pch=15)
   
}

In [ ]:
plot.correlations <- function(chain.df,mfrow){
    options(repr.plot.width=15, repr.plot.height=15)
    par(mfrow=mfrow)
    for(i in 1:(length(chain.df)-1)){
        for (j in (i+1):(length(chain.df))){
            points.i <- chain.df[,i]
            points.j <- chain.df[,j]
            plot(points.i,points.j, xlab=names(chain.df)[i], ylab=names(chain.df)[j], col="skyblue",
                cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)
  
        }
    }
}

In [ ]:
plot.autoc <- function(chain.df,mfrow){
    options(repr.plot.width=10, repr.plot.height=7)
    par(mfrow=mfrow)
    for (param in names(chain.df)){
        single.chain <- chain.df[param]
        acf.param <- acf(single.chain, plot=TRUE,col="palevioletred",relative=TRUE)

    }    
    
}

In [ ]:
plot.stats <- function(chain,param,leg=TRUE, digits=3, return.verbose=FALSE){

    param.chain <- chain[,param]
    h <- hist(param.chain,plot=FALSE)
    hdf <- HPDinterval(param.chain, 0.68)
    color.cut <- cut(h$breaks,c(-Inf, hdf[,"lower"], hdf[,"upper"], +Inf))
    mode <- format(h$mids[which.max(h$density)], digits=digits)
    
    kde <- density(param.chain)
    
    y.lim <- max(max(kde$y),max(h$density))
    
    if(return.verbose == TRUE){
        out <- c(mode,hdf[,"lower"], hdf[,"upper"])
        return (out)
    }
    else{
        #histogram and mode
        plot(h, freq=FALSE,col=c("lightskyblue1","lightskyblue3","lightskyblue1")[color.cut],
             xlab=paste("value of ",param), main=paste(param, " chain"),
            cex.lab=2, cex.axis=2, cex.main=2, cex.sub=2,
                  ylim=c(0,y.lim))

        #KDE
        points(kde, type="l", col="navy", lwd=3)

        #mean and mode
        mean <- format(mean(param.chain), digits=digits)
        abline(v = mean, col="darkgoldenrod1", lwd=3, lty=2)
        abline(v = mode, col="palevioletred", lwd=3, lty=3)

        if (leg != FALSE){
            #op <- par(cex = 0.8)
            legend("topright", c("Normalized histogram", "KDE","mean ", "mode", "68% HDPinterval"), 
               col=c("lightskyblue1","navy","darkgoldenrod1", "palevioletred", "lightskyblue3"),pch=15,bty="n",cex=1.8, 
                   inset=c(0.15,0),y.intersp=0.3, x.intersp=0.1)
        }
    }
}

    

In [ ]:
plot.stats.2 <- function(chain,param,leg=TRUE, digits=3, return.verbose=FALSE){

    param.chain <- chain[,param]
    h <- hist(param.chain,plot=FALSE)
    hdf <- HPDinterval(param.chain, 0.68)
    color.cut <- cut(h$breaks,c(-Inf, hdf[,"lower"], hdf[,"upper"], +Inf))
    mode <- format(h$mids[which.max(h$density)], digits=digits)
    
    kde <- density(param.chain)
    
    y.lim <- max(max(kde$y),max(h$density))
    
    if(return.verbose == TRUE){
        out <- c(mode,hdf[,"lower"], hdf[,"upper"])
        return (out)
    }
    else{
        #histogram and mode
        plot(h, freq=FALSE,col=c("lightskyblue1","lightskyblue3","lightskyblue1")[color.cut],
             xlab=paste("value of ",param), main=paste(param, " chain"),
            cex.lab=2, cex.axis=2, cex.main=2, cex.sub=2,
                  ylim=c(0,y.lim))

        #KDE
        points(kde, type="l", col="navy", lwd=3)

        #mean and mode
        mean <- format(mean(param.chain), digits=digits)
        abline(v = mean, col="darkgoldenrod1", lwd=3, lty=2)
        abline(v = mode, col="palevioletred", lwd=3, lty=3)

        if (leg != FALSE){
            #op <- par(cex = 0.8)
            legend("topright", c("Normalized histogram", "KDE","mean","mode", "68% HDPinterval"), 
               col=c("lightskyblue1","navy","darkgoldenrod1", "palevioletred", "lightskyblue3"),
                   pch=15,bty="n",cex=1.8, inset=c(0.15,0),y.intersp=0.1,x.intersp=0.1)
        }
    }
}

    

In [ ]:
mode.lower.upper.func <- function(chain){
    
    chain.df <-as.data.frame(as.mcmc(chain))
    params.names <- names(chain.df)
    
    n.params <- length(params.names)
    iterations <- n.params
    variables <- 3 #mode, lower, upper

    mode.lower.upper <- matrix(ncol=variables, nrow=iterations)

    for(i in 1:iterations){
        mode.lower.upper[i,] <- plot.stats(chain, params.names[i], leg=FALSE, digits=5,return.verbose=TRUE)
    }
    
    mode.lower.upper.df <- as.data.frame(mode.lower.upper,stringsAsFactors=FALSE)
    colnames(mode.lower.upper.df) <- c("mode","lower","upper")
    row.names(mode.lower.upper.df) <- params.names
    mode.lower.upper.df[] <- lapply(mode.lower.upper.df, type.convert, as.is = TRUE)
    return(mode.lower.upper.df)
    
}

In [ ]:
testz<-function(val.teo,val.exp,err.exp){
    return((abs(val.teo-val.exp))/err.exp)
    }

## Integration Functions

In [ ]:
Routti.Prussin.peak.fix<- function(z)
    {
    return(Routti.Prussin.peak(z,J=best.params[1],A=best.params[2],mu=best.params[6],sigma=best.params[8]))
    }

In [ ]:
Routti.Prussin.peak<-function(z,J,A,mu,sigma)
    {
    ifelse(z < mu - J,  A/(sigma)*exp(J*(2*z - 2*mu + J)/(2*sigma*sigma)),
           A/(sigma)*exp((-(z-mu)*(z-mu))/(2*sigma*sigma)))
    }

# COBALT 1 E = 1.17 MeV

In [ ]:
summary.table["Co1",]

In [ ]:
rangeCo1 = dataset[!(dataset$ADC_channel<=summary.table["Co1","range.gauss.min"]) & (dataset$ADC_channel<=summary.table["Co1","range.gauss.max"]),]
rangeCo1$ADC_channel <- rangeCo1$ADC_channel - summary.table["Co1","argmax.channel"]
head(rangeCo1)

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)
par(mfrow=c(1,2))
plot(rangeCo1$ADC_channel+summary.table["Co1","argmax.channel"],rangeCo1$Counts,type = "p", col='red', main = 'Co1 E = 1.17 MeV', xlab = 'ADC channel [#]', ylab = "Counts [#]",cex.main = 1.5)
plot(rangeCo1$ADC_channel+summary.table["Co1","argmax.channel"],log(rangeCo1$Counts), type = "p", col='red', main = 'Co1 E = 1.17 MeV', xlab = 'ADC channel [#]', ylab = "Log(Counts) [#]",cex.main = 1.5)

We choose as a generative model:

$\text{Background}(x |\alpha,\text{offset}) = 
    \exp[\tan[\alpha]x + \text{offset}]
$


$\text{Signal}(x |\text{N},\mu,\sigma) = \frac{N}{\sqrt{2 \pi}\sigma} \exp \left[ -\frac{(x-\mu)^2}{2\sigma^2}\right] 
$

In [ ]:
cat("model{
    #data likelihood
    for (i in 1:length(x)){

        points[i] <- N/sqrt(2*pi*sigma*sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)) +  #normal generative model
                     exp(tan(slope.alpha)*(x[i]-mu) + offset)  #line generative model
        
        y[i] ~ dpois(points[i]) 


    }

    #priors for the generative model

    N ~ dunif(50000,400000)            
    mu ~ dunif(-50,50)        
    sigma ~ dgamma(0.0085,0.001)             
    offset ~ dunif(0,500)        
    slope.alpha ~ dunif(-pi/2 , 0)  

    #true value of mu
    mu.t <- mu + argmax.channel

}", file="Bmodel.cobaltum1.txt")

model.Co1 <- "Bmodel.cobaltum1.txt"

In [ ]:
#we make an estimate of the true values (via mathematica or looking at the plot)
#TRUE values are unkown

init.val.Co1.1 <- list(N =300000,mu = 0,sigma =15,slope.alpha=-0.001,offset=10,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Co1.2 <- list(N =350000,mu = 10,sigma =15,slope.alpha=0,offset=50,
                    .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Co1.3 <- list(N =200000,mu = -25,sigma =10,slope.alpha=-pi/4,offset=20,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Co1 <- list(init.val.Co1.1 , init.val.Co1.2 , init.val.Co1.3 )

data.Co1 <- NULL
data.Co1$x <- rangeCo1$ADC_channel
data.Co1$y <- rangeCo1$Counts
data.Co1$pi <- pi
data.Co1$argmax.channel <- summary.table["Co1","argmax.channel"]

In [ ]:
#reading the model
jmCo1 <- jags.model(model.Co1, data.Co1, init.val.Co1, n.adapt=0, n.chains=3) #le chains sono utili le terrei a prescindere
update(jmCo1,1000)
chain.Co1 <- coda.samples(jmCo1 , c("N","mu.t", "sigma","slope.alpha","offset"), n.iter=8000, thin = 10) 

In [ ]:
#Gelman and Rubin's convergence diagnostic:ratio between variance between the chains and variance within a chain
gelman.diag(chain.Co1)$psrf[,1]
cat(paste("Total chain R value:",gelman.diag(chain.Co1)$mpsrf))

In [ ]:
params=summary(chain.Co1)$statistics[,1:2]
params

In [ ]:
total.chain.Co1 <- mcmc(do.call(rbind, chain.Co1),thin=thin(chain.Co1)) #combines the three chains

In [ ]:
par(mfrow=c(3,2),mar=c(5,5.5,4,0.7))
options(repr.plot.width=15, repr.plot.height=25)
chain.Co1.df <- as.data.frame ( as.mcmc(total.chain.Co1) )

for (p in names(chain.Co1.df)){
    plot.stats(total.chain.Co1, p, leg=TRUE, digits=7)
}

VISUALIZATION OF THE CHAINS

In [ ]:
options(repr.plot.width=10, repr.plot.height=15)
plot(chain.Co1,cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF THE AUTOCORRELATION PARAMETRS (IN ORDER TO ADJUST THINNING PARAMETER)

In [ ]:
autocorr.plot(total.chain.Co1, cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF THE CORRELATION BETWEEN VARIABLES

In [ ]:
plot.correlations(chain.Co1.df, c(3,4))

## LOGPosterior

In [ ]:
y=data.Co1$y
x=data.Co1$x 
parametri <- as.data.frame(as.mcmc(total.chain.Co1))



best.params <- c(0,0,0,0,0)
LogPostBest <- -Inf

for (j in 1:dim(total.chain.Co1)[1]){

    
    N=parametri$N[j]
    mu=parametri$mu[j] - summary.table["Co1","argmax.channel"]
    sigma=parametri$sigma[j]
    slope.alpha=parametri$slope.alpha[j]
    offset=parametri$offset[j]

  
    #LogLikelihood

    LogL <- 0

    for (i in 1:length(y)){

        point <- N/sqrt(2*pi*sigma*sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)) + #+  #normal generative model
                                exp(tan(slope.alpha)*(x[i]-mu) + offset)  #line generative model

        p.y <- dpois(y[i], point) 

        LogL <- LogL + log(p.y)

    }


    #LogPrior
    p.N <- dunif(N,50000,300000)           
    p.mu <- dunif(mu,-50,50)        
    p.sigma <- dgamma(sigma,60,7)          
    p.slope.alpha <- dunif(slope.alpha,-pi/2,0)  
    p.offset <- dunif(offset,0,50) 
    
    LogPrior <- log(p.N) + log(p.mu) + log(p.slope.alpha) + log(p.offset) + log(p.sigma)


    #LogPost
    LogPost <- LogPrior + LogL

    #cat(paste("LogL:",LogL,"\n"))
    #cat(paste("LogPost:",LogPost,"\n"))

    
    
    #check for maximum of the posterior
    if(LogPost > LogPostBest){
        LogPostBest <- LogPost
        best.params <- c(N, mu, offset, sigma, slope.alpha)
    }
    
}
best.params[2] <- best.params[2] + summary.table["Co1","argmax.channel"]


In [ ]:
cat(best.params)

In [ ]:
###confronto tra media, moda e originale
params=summary(chain.Co1)$statistics[,1]
mlu <- mode.lower.upper.func(total.chain.Co1)
mlu



extracted <-  sapply(data.Co1$x+summary.table["Co1","argmax.channel"],Gauss.exp.func,slope=best.params[5],offset=best.params[3],N=best.params[1],mu=best.params[2],sigma=best.params[4])
extracted.moda <-  sapply(data.Co1$x+summary.table["Co1","argmax.channel"],Gauss.exp.func, slope=mlu[5,1],offset=mlu[3,1],N=mlu[1,1],mu=mlu[2,1],sigma=mlu[4,1])
extracted.mean <-  sapply(data.Co1$x+summary.table["Co1","argmax.channel"],Gauss.exp.func,slope=params[5],offset=params[3],N=params[1],mu=params[2],sigma=params[4])



plot.func.total.1(data.Co1$x+summary.table["Co1","argmax.channel"],data.Co1$y,extracted,extracted.moda,extracted.mean,title=("Cobalt E = 1.17 MeV"))

# COBALT 2 E = 1.33 MeV

In [ ]:
summary.table["Co2",]

In [ ]:
rangeCo2 = dataset[!(dataset$ADC_channel<=summary.table["Co2","range.gauss.min"]) & (dataset$ADC_channel<=summary.table["Co2","range.gauss.max"]),]
rangeCo2$ADC_channel <- rangeCo2$ADC_channel - summary.table["Co2","argmax.channel"]
head(rangeCo2)

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)
par(mfrow=c(1,2))
plot(rangeCo2$ADC_channel + summary.table["Co2","argmax.channel"],rangeCo2$Counts, ylim=c(0,300),type = "p", col='red', main = 'Co2 E = 1.33 MeV', xlab = 'ADC channel [#]', ylab = "Counts [#]",cex.main = 1.5)
#points(rangeCo2$ADC_channel + summary.table["Co2","argmax.channel"],exp(tan(params[5])*(rangeCo2$ADC_channel) + params[3]))
plot(rangeCo2$ADC_channel + summary.table["Co2","argmax.channel"],log(rangeCo2$Counts), type = "p", col='red', main = 'Co2 E = 1.33 MeV', xlab = 'ADC channel [#]', ylab = "log(Counts) [#]",cex.main = 1.5)

In [ ]:
cat("model{
    #data likelihood
    for (i in 1:length(x)){

        points[i] <- N/sqrt(2*pi*sigma*sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)) + #+  #normal generative model
                            exp(tan(slope.alpha)*(x[i]-mu) + offset)  #line generative model
                        
        y[i] ~ dpois(points[i]) 


    }
    #priors for the generative model

    N ~ dunif(50000,300000)           
    mu ~ dunif(-50,50)        
    sigma ~ dgamma(0.0085,0.001)            
    slope.alpha ~ dunif(-pi/2,0)  
    offset ~ dunif(0,50)       

    #true value of mu
    mu.t <- mu + argmax.channel

   

}", file="Bmodel.cobaltum2.pois.txt")
model.Co2 <- "Bmodel.cobaltum2.pois.txt"

In [ ]:
#MODEL SET UP: INITIAL VALUES AND DATA PREPARATION

#we make an estimate of the true values looking at the plot
#TRUE values are unkown


init.val.Co2.1 <- list(N =100000,mu = 12,sigma =15, slope.alpha=-0.01,offset=2,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Co2.2 <- list(N =150000,mu = -30,sigma =7,slope.alpha=-1,offset=8,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Co2.3 <- list(N =210000,mu = 3,sigma =10,slope.alpha=-pi/3,offset=20,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Co2 <- list(init.val.Co2.1 , init.val.Co2.2 , init.val.Co2.3 )

data.Co2 <- NULL
data.Co2$x <- rangeCo2$ADC_channel
data.Co2$y <- rangeCo2$Counts
data.Co2$pi <- pi
data.Co2$argmax.channel <- summary.table["Co2","argmax.channel"]

In [ ]:
#reading the model
jmCo2 <- jags.model(model.Co2, data.Co2, inits=init.val.Co2, n.adapt=0, n.chains=3)
update(jmCo2, 1000)
chain.Co2 <- coda.samples(jmCo2 , c("N","mu.t", "sigma","slope.alpha","offset"),n.iter=8000, thin=10)#,"slope.alpha","offset"), n.iter=5000, thin=1) 

In [ ]:
#Gelman and Rubin's convergence diagnostic:ratio between variance between the chains and variance within a chain
gelman.diag(chain.Co1)$psrf[,1]
cat(paste("Total chain R value:",gelman.diag(chain.Co1)$ mpsrf))

In [ ]:
params=summary(chain.Co2)$statistics[,1:2]
params

In [ ]:
total.chain.Co2 <- mcmc(do.call(rbind, chain.Co2),thin=thin(chain.Co2)) #combines the three chains

In [ ]:
par(mfrow=c(3,2),mar=c(5,5.5,4,0.7))
options(repr.plot.width=15, repr.plot.height=25)
chain.Co2.df <- as.data.frame ( as.mcmc( total.chain.Co2 ) )

for (p in names(chain.Co2.df)){
    plot.stats(total.chain.Co2, p, leg=TRUE, digits=8)
}


VISUALIZATION OF THE CHAIN



In [ ]:
options(repr.plot.width=10, repr.plot.height=15)
plot(chain.Co2,cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZTION OF AUTOCORRELATION

In [ ]:
autocorr.plot(chain.Co2.df,cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF CORRELATION BETWEEN VARIABLES

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
plot.correlations(chain.Co2.df, c(3,4))

## LOGPosterior

In [ ]:
y=data.Co2$y
x=data.Co2$x 
parametri <- as.data.frame(as.mcmc(total.chain.Co2))



best.params <- c(0,0,0,0,0)
LogPostBest <- -Inf

for (j in 1:dim(total.chain.Co2)[1]){

    
    N=parametri$N[j]
    mu=parametri$mu[j] - summary.table["Co2","argmax.channel"]
    sigma=parametri$sigma[j]
    slope.alpha=parametri$slope.alpha[j]
    offset=parametri$offset[j]

  
    #LogLikelihood

    LogL <- 0

    for (i in 1:length(y)){

        point <- N/sqrt(2*pi*sigma*sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)) + #+  #normal generative model
                                exp(tan(slope.alpha)*(x[i]-mu) + offset)  #line generative model

        p.y <- dpois(y[i], point) 

        LogL <- LogL + log(p.y)

    }


    #LogPrior
    p.N <- dunif(N,50000,300000)           
    p.mu <- dunif(mu,-50,50)        
    p.sigma <- dgamma(sigma,60,7)          
    p.slope.alpha <- dunif(slope.alpha,-pi/2,0)  
    p.offset <- dunif(offset,0,50) 
    
    LogPrior <- log(p.N) + log(p.mu) + log(p.slope.alpha) + log(p.offset) + log(p.sigma)


    #LogPost
    LogPost <- LogPrior + LogL

    #cat(paste("LogL:",LogL,"\n"))
    #cat(paste("LogPost:",LogPost,"\n"))

    
    
    #check for maximum of the posterior
    if(LogPost > LogPostBest){
        LogPostBest <- LogPost
        best.params <- c(N, mu, offset, sigma, slope.alpha)
    }
    
}
best.params[2] <- best.params[2] + summary.table["Co2","argmax.channel"]
best.params

In [ ]:
###confronto tra media, moda e originale
params=summary(chain.Co2)$statistics[,1]
mlu <- mode.lower.upper.func(total.chain.Co2)
mlu



extracted <-  sapply(data.Co2$x+summary.table["Co2","argmax.channel"],Gauss.exp.func,slope=best.params[5],offset=best.params[3],N=best.params[1],mu=best.params[2],sigma=best.params[4])
extracted.moda <-  sapply(data.Co2$x+summary.table["Co2","argmax.channel"],Gauss.exp.func, slope=mlu[5,1],offset=mlu[3,1],N=mlu[1,1],mu=mlu[2,1],sigma=mlu[4,1])
extracted.mean <-  sapply(data.Co2$x+summary.table["Co2","argmax.channel"],Gauss.exp.func,slope=params[5],offset=params[3],N=params[1],mu=params[2],sigma=params[4])



plot.func.total.1(data.Co2$x+summary.table["Co2","argmax.channel"],data.Co2$y,extracted,extracted.moda,extracted.mean,title=("Cobalt E = 1.33 MeV"))

# DOUBLE ABSORPTION 

In [ ]:
summary.table["DA",]

In [ ]:
summary.table["DA","argmax.channel"] <- summary.table["DA","argmax.channel"]

In [ ]:
rangeDA = dataset[!(dataset$ADC_channel<=summary.table["DA","range.gauss.min"]) & (dataset$ADC_channel<=summary.table["DA","range.gauss.max"]),]

In [ ]:
rangeDA$ADC_channel <- rangeDA$ADC_channel - summary.table["DA","argmax.channel"]

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)
par(mfrow=c(1,2))
plot(rangeDA$ADC_channel + summary.table["DA","argmax.channel"],rangeDA$Counts, type = "p", col='red', main = "Double absorption E = 2.506 MeV", xlab = 'ADC channel [#]', ylab = "Counts [#]",cex.main = 1.5)
#points(rangeDA$ADC_channel + summary.table["DA","argmax.channel"],exp(tan(params[5])*(rangeDA$ADC_channel) + params[3]))
plot(rangeDA$ADC_channel + summary.table["DA","argmax.channel"],log(rangeDA$Counts), type = "p", col='red', main = "Double absorption E = 2.506 MeV", xlab = 'ADC channel [#]', ylab = "log(Counts) [#]",cex.main = 1.5)

In [ ]:
cat("model{
    #data likelihood
    for (i in 1:length(x)){

        points[i] <- N/sqrt(2*pi*sigma*sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)) + #+  #normal generative model
                            exp(tan(slope.alpha)*(x[i]-mu) + offset)  #line generative model
                        
        y[i] ~ dpois(points[i]) 


    }

    #priors for the generative model

    N ~ dunif(5000,50000)           
    mu ~ dunif(-100,100)        
    sigma ~ dgamma(0.0085,0.001)          
    slope.alpha ~ dunif(-pi/4,0)  
    offset ~ dunif(0,100)       

    mu.t <- mu + argmax.channel

   

}", file="Bmodel.double.txt")
model.DA <- "Bmodel.double.txt"

In [ ]:
#we make an estimate of the true values looking at the plot
#TRUE values are unkown


init.val.DA.1 <- list(N =10000,mu = 0,sigma =40,slope.alpha=-0.003,offset=10,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.DA.2 <- list(N =20000,mu = -10,sigma =50,slope.alpha=-0.013,offset=5,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.DA.3 <- list(N =5000,mu = 10,sigma =20,slope.alpha=-0.1,offset=40,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.DA <- list(init.val.DA.1 , init.val.DA.2 , init.val.DA.3 )

data.DA <- NULL
data.DA$x <- rangeDA$ADC_channel
data.DA$y <- rangeDA$Counts
data.DA$pi <- pi
data.DA$argmax.channel <- summary.table["DA","argmax.channel"]

In [ ]:
#reading the model
jmDA <- jags.model(model.DA, data.DA, inits=init.val.DA, n.chain=3,n.adapt=0)
update(jmDA,1000)
chain.DA <- coda.samples(jmDA , c("N","mu.t", "sigma","slope.alpha","offset"), n.iter=10000, thin=15) 

In [ ]:
#Gelman and Rubin's convergence diagnostic:ratio between variance between the chains and variance within a chain
gelman.diag(chain.DA)$psrf[,1]
cat(paste("Total chain R value:",gelman.diag(chain.DA)$ mpsrf))

In [ ]:
params=summary(chain.DA)$statistics[,1]
summary(chain.DA)$statistics[,1:2]

In [ ]:
total.chain.DA <- mcmc(do.call(rbind, chain.DA),thin=thin(chain.DA)) #combines the three chains

In [ ]:
par(mfrow=c(3,2),mar=c(5,5.5,4,0.7))
options(repr.plot.width=15, repr.plot.height=25)


chain.DA.df <- as.data.frame ( as.mcmc( total.chain.DA ) )

for (p in names(chain.DA.df)){
    plot.stats(total.chain.DA, p, leg=TRUE, digits=7)
}

VISUALIZATION OF THE CHAIN

In [ ]:
options(repr.plot.width=10, repr.plot.height=15)
plot(chain.DA,cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF AUTOCORRELATION

In [ ]:
autocorr.plot(chain.DA.df,cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF CORRELATION BETWEEN VARIABLES

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
plot.correlations(chain.DA.df, c(3,4))

## LOGPosterior

In [ ]:
y=data.DA$y
x=data.DA$x 
parametri <- as.data.frame(as.mcmc(total.chain.DA))

best.params <- c(0,0,0,0,0)
LogPostBest <- -Inf

for (j in 1:dim(total.chain.DA)[1]){

    
    N=parametri$N[j]
    mu=parametri$mu[j] - summary.table["DA","argmax.channel"]
    sigma=parametri$sigma[j]
    slope.alpha=parametri$slope.alpha[j]
    offset=parametri$offset[j]

  
    #LogLikelihood

    LogL <- 0

    for (i in 1:length(y)){

        point <- N/sqrt(2*pi*sigma*sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)) + #+  #normal generative model
                                exp(tan(slope.alpha)*(x[i]-mu) + offset)  #line generative model

        p.y <- dpois(y[i], point) 

        LogL <- LogL + log(p.y)

    }


    #LogPrior
    p.N <- dunif(N,5000,50000)           
    p.mu <- dunif(mu,-100,100)        
    p.sigma <- dgamma(sigma,60,7)          
    p.slope.alpha <- dunif(slope.alpha,-pi/4,0)  
    p.offset <- dunif(offset,0,100)      

    LogPrior <- log(p.N) + log(p.mu) + log(p.slope.alpha) + log(p.offset) + log(p.sigma)


    #LogPost
    LogPost <- LogPrior + LogL

    #cat(paste("LogL:",LogL,"\n"))
    #cat(paste("LogPost:",LogPost,"\n"))

    
    
    #check for maximum of the posterior
    if(LogPost > LogPostBest){
        LogPostBest <- LogPost
        best.params <- c(N, mu, offset, sigma, slope.alpha)
    }
    
}
best.params[2] <- best.params[2] + summary.table["DA","argmax.channel"]
best.params

In [ ]:
###confronto tra media, moda e originale
params=summary(chain.DA)$statistics[,1]
mlu <- mode.lower.upper.func(total.chain.DA)
mlu



extracted <-  sapply(data.DA$x+summary.table["DA","argmax.channel"],Gauss.exp.func,slope=best.params[5],offset=best.params[3],N=best.params[1],mu=best.params[2],sigma=best.params[4])
extracted.moda <-  sapply(data.DA$x+summary.table["DA","argmax.channel"],Gauss.exp.func, slope=mlu[5,1],offset=mlu[3,1],N=mlu[1,1],mu=mlu[2,1],sigma=mlu[4,1])
extracted.mean <-  sapply(data.DA$x+summary.table["DA","argmax.channel"],Gauss.exp.func,slope=params[5],offset=params[3],N=params[1],mu=params[2],sigma=params[4])



plot.func.total.1(data.DA$x+summary.table["DA","argmax.channel"],data.DA$y,extracted,extracted.moda,extracted.mean,title=("Double absorption E = 2.506 MeV"))

# AMERICIUM

In [ ]:
summary.table["Am",]

In [ ]:
rangeAm = dataset[!(dataset$ADC_channel<=summary.table["Am","range.gauss.min"]) & (dataset$ADC_channel<=summary.table["Am","range.gauss.max"]),]
rangeAm <- data.frame(rangeAm["Counts"], rangeAm["ADC_channel"])
head(rangeAm)

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)
par(mfrow=c(1,2))
plot(rangeAm$ADC_channel,rangeAm$Counts, type = "p", col='red',main = 'Am E =0.059 MeV', xlab = 'ADC channel [#]', ylab = "Counts [#]",cex.main = 1.5)
plot(rangeAm$ADC_channel,log(rangeAm$Counts), type = "p", col='red',main = 'Am E =0.059 MeV', xlab = 'ADC channel [#]', ylab = "log(Counts) [#]",cex.main = 1.5)

## GAUSSIAN

We choose as a generative model:


$\text{Background}(x |c.1,c.2,\mu_{\sigma},\text{offset}) = 
    \frac{c.1}{1+\exp \left[ c.2(x-\mu_{s})\right] } + \text{offset}
$


$\text{Signal}(x |\text{N},\mu,\sigma) = \frac{N}{\sqrt{2 \pi}\sigma} \exp \left[ -\frac{(x-\mu)^2}{2\sigma^2}\right] 
$

In [ ]:
cat("model{
    #data likelihood
    for (i in 1:length(x)){

        points[i] <- N/sqrt(2*pi*sigma*sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)) +
                     c.1/(1+exp(c.2*(x[i]-c.mu))) + offset
                        
        y[i] ~ dpois(points[i])


    }
    #priors for the generative model

    N ~ dunif(50000,500000)           
    mu ~ dunif(150,165)        
    sigma ~ dgamma(0.0085,0.001)            
    c.1 ~ dunif(400,3000)          
    c.2 ~ dunif(0,1)      
    c.mu ~ dnorm(mu,1)          
    offset ~ dunif(0,2000)         

    

}", file="BUGS_model_Am_S1.txt")
model.Am.S1 <- "BUGS_model_Am_S1.txt"


In [ ]:
#SOLO PER PROVARE
init.val.Am.S1.1 <- list(N =100000,mu = 154,sigma =7,c.mu = 156, c.1=1000, c.2 = 0.5,offset=2000,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Am.S1.2 <- list(N =170000,mu = 155,sigma =10,c.mu = 150, c.1=2000, c.2 = 0.1, offset=1200,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Am.S1.3 <- list(N =185000,mu = 160,sigma =15, c.mu=160, c.1=1200, c.2 = 0.6, offset=800,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Am.S1 <- list(init.val.Am.S1.1 , init.val.Am.S1.2 , init.val.Am.S1.3 )


data.Am.S1<- NULL
data.Am.S1$x <- rangeAm$ADC_channel
data.Am.S1$y <- rangeAm$Counts
data.Am.S1$pi <- pi

In [ ]:
jm.Am.S1 <- jags.model(model.Am.S1, data.Am.S1, init.val.Am.S1,n.adapt=0,n.chains=3)
update(jm.Am.S1, 1000)
chain.Am.S1 <- coda.samples(jm.Am.S1 , c("N","mu", "sigma","c.1","c.2","c.mu","offset"), n.iter=10000, thin=15) 

In [ ]:
#Gelman and Rubin's convergence diagnostic:ratio between variance between the chains and variance within a chain
gelman.diag(chain.Am.S1)$psrf[,1]
cat(paste("Total chain R value:",gelman.diag(chain.Am.S1)$ mpsrf))

In [ ]:
params=summary(chain.Am.S1)$statistics[,1:2]
params

In [ ]:
total.chain.Am.S1 <- mcmc(do.call(rbind, chain.Am.S1),thin=thin(chain.Am.S1)) #combines the three chains

In [ ]:
par(mfrow=c(4,2),mar=c(5,5.5,4,0.7))
options(repr.plot.width=15, repr.plot.height=30)

chain.Am.S1.df <- as.data.frame ( as.mcmc( total.chain.Am.S1 ) )

for (p in names(chain.Am.S1.df)){
    plot.stats.2(total.chain.Am.S1, p, leg=TRUE, digits=8)
}


VISUALIZATION OF THE CHAIN

In [ ]:
#VISUALIZATION OF THE CHAIN

options(repr.plot.width=10, repr.plot.height=15)
plot(chain.Am.S1,cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF AUTOCORRELATION

In [ ]:
autocorr.plot(chain.Am.S1.df,mfrow=c(3,3),cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF CORRELATION BETWEEN PARAMETERS

In [ ]:
plot.correlations(chain.Am.S1.df,c(3,3))

### LOGPosterior

In [ ]:
y=data.Am.S1$y
x=data.Am.S1$x 
parametri <- as.data.frame(as.mcmc(total.chain.Am.S1))



best.params <- c(0,0,0,0,0)
LogPostBest <- -Inf

for (j in 1:dim(total.chain.Am.S1)[1]){

    
    N=parametri$N[j]
    mu=parametri$mu[j] 
    sigma=parametri$sigma[j]
    c.1=parametri$c.1[j]
    c.2=parametri$c.2[j]
    c.mu=parametri$c.mu[j]
    offset=parametri$offset[j]

  
    #LogLikelihood

    LogL <- 0

    for (i in 1:length(y)){

        point <- N/sqrt(2*pi*sigma*sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)) + #+  #normal generative model
                                c.1/(1+exp(c.2*(x[i]-c.mu))) + offset  #line generative model

        p.y <- dpois(y[i], point) 

        LogL <- LogL + log(p.y)

    }
    

    #LogPrior
    p.N <- dunif(N,50000,500000)           
    p.mu <- dunif(mu,150,165)        
    p.sigma <- dgamma(sigma,6,2)          
    p.c.1 <- dunif(c.1,400,3000)
    p.c.2 <- dunif(c.2,0,1) 
    p.c.mu <- dnorm(c.mu,mu,1)
    p.offset <- dunif(offset,0,2000) 
 
    LogPrior <- log(p.N) + log(p.mu) + log(p.sigma) + log(p.c.1) + 
                log(p.sigma) + log(p.c.2) + log(p.c.mu) + log(p.offset)


    #LogPost
    LogPost <- LogPrior + LogL

    #cat(paste("LogL:",LogL,"\n"))
    #cat(paste("LogPost:",LogPost,"\n"))

    
    
    #check for maximum of the posterior
    if(LogPost > LogPostBest){
        LogPostBest <- LogPost
        best.params <- c(N, c.1, c.2, c.mu, mu, offset, sigma)
    }
    
}



In [ ]:
cat(best.params)

In [ ]:
###confronto tra media, moda e originale
params=summary(chain.Am.S1)$statistics[,1]
mlu <- mode.lower.upper.func(total.chain.Am.S1)
mlu



extracted <-  sapply(data.Am.S1$x,Gauss.Sig.func, N=best.params[1],c.1=best.params[2],c.2=best.params[3],c.mu=best.params[4],mu=best.params[5],offset=best.params[6],sigma=best.params[7])
extracted.moda <-  sapply(data.Am.S1$x, Gauss.Sig.func,N=mlu[1,1],c.1=mlu[2,1],c.2=mlu[3,1],c.mu=mlu[4,1],mu=mlu[5,1],offset=mlu[6,1],sigma=mlu[7,1])
extracted.mean <-  sapply(data.Am.S1$x, Gauss.Sig.func,N=params[1],c.1=params[2],c.2=params[3],c.mu=params[4],mu=params[5],offset=params[6],sigma=params[7])


plot.func.total.1(data.Am.S1$x,data.Am.S1$y,extracted,extracted.moda,extracted.mean,title=("Americium (Gaussian + Sigmoid) E = 0.059 MeV"))

## Routti and Prussin Peak-Shape Function (1969) [1] 

$\text{Background}(x |c.1,c.2,\mu_{\sigma},\text{offset}) = 
    \frac{c.1}{1+\exp \left[ c.2(x-\mu_{s})\right] } + \text{offset}
$


$\text{Signal}(x |\text{A},J,\mu,\sigma) = \begin{cases}
    A \exp\left[ \frac{J (2x-2\mu+J)}{2\sigma^2} \right]& \mbox{if } x < \mu-J \\
   A \exp\left[ -\frac{(x-\mu)^2}{2\sigma^2} \right]   & \mbox{if } x > \mu-J
\end{cases}
$




$\text{Total}(x |\text{A},J,\mu,\sigma,c.1,c.2,\mu_{\sigma},\text{offset}) = \begin{cases}
    A \exp\left[ \frac{J (2x-2\mu+J)}{2\sigma^2} \right]+ \frac{c.1}{1+\exp \left[ c.2(x-\mu_{s})\right] } + \text{offset}& \mbox{if } x < \mu-J \\
   A \exp\left[ -\frac{(x-\mu)^2}{2\sigma^2} \right] + \frac{c.1}{1+\exp \left[ c.2(x-\mu_{s})\right] } + \text{offset} & \mbox{if } x > \mu-J
\end{cases}
$

In [ ]:
cat("model{
    #data likelihood
    for (i in 1:length(x)){

                        
    points[i] <- step(mu - J - x[i])*(c.1/(1 + exp(c.2*(x[i] - c.mu))) + offset +  (A/sigma)*exp(J*(2*x[i] - 2*mu + J)/(2*sigma*sigma))) +
                 step(x[i] - mu + J)*(c.1/(1 + exp(c.2*(x[i] - c.mu))) + offset + (A/sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)))

    y[i] ~ dpois(points[i])

    }
    #priors for the generative model
    
    A ~ dunif(50000,500000)
    mu ~ dunif(150,170)
    sigma ~ dgamma(0.0085,0.001)
    c.1 ~ dunif(1000,4000)   
    c.2 ~ dunif(0,0.6)  
    c.mu ~ dnorm(mu,1/4)
    J ~ dunif(0,100)
    offset ~ dnorm(500,2000) 

}", file="Bmodel.americium.txt")

model.Am <- "Bmodel.americium.txt"

In [ ]:
#SOLO PER PROVARE
init.val.Am.1 <- list(A =110000,mu = 154,sigma =7,c.mu = 156, c.1=1000, c.2 = 0.5,offset=2000, J=5,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Am.2 <- list(A =150000,mu = 155,sigma =10,c.mu = 150, c.1=2000, c.2 = 0.1, offset=1200, J=50,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Am.3 <- list(A =300000,mu = 160,sigma =15, c.mu=160, c.1=1200, c.2 = 0.6, offset=800, J=28,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Am <- list(init.val.Am.1 , init.val.Am.2 , init.val.Am.3 )


data.Am<- NULL
data.Am$x <- rangeAm$ADC_channel
data.Am$y <- rangeAm$Counts

In [ ]:
jm.Am <- jags.model(model.Am, data.Am, init.val.Am,n.adapt=0,n.chains=3)
update(jm.Am, 5000)
chain.Am <- coda.samples(jm.Am , c("A","mu", "sigma","c.1","c.2","c.mu","offset","J"), n.iter=50000, thin=20) 

In [ ]:
#Gelman and Rubin's convergence diagnostic:ratio between variance between the chains and variance within a chain
gelman.diag(chain.Am)$psrf[,1]
cat(paste("Total chain R value:",gelman.diag(chain.Am)$ mpsrf))

In [ ]:
params=summary(chain.Am)$statistics[,1:2]
params

In [ ]:
total.chain.Am <- mcmc(do.call(rbind, chain.Am),thin=thin(chain.Am)) #combines the three chains

In [ ]:
par(mfrow=c(4,2),mar=c(5,5.5,4,0.7))
options(repr.plot.width=15, repr.plot.height=30)

chain.Am.df <- as.data.frame ( as.mcmc( total.chain.Am ) )

for (p in names(chain.Am.df)){
    plot.stats.2(total.chain.Am, p, leg=TRUE, digits=8)
}


VISUALIZATION OF THE CHAIN

In [ ]:
#VISUALIZATION OF THE CHAIN

options(repr.plot.width=10, repr.plot.height=15)
plot(chain.Am,cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF AUTOCORRELATION

In [ ]:
autocorr.plot(chain.Am.df,mfrow=c(3,3),cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF CORRELATION BETWEEN PARAMETERS

In [ ]:
plot.correlations(chain.Am.df,c(3,3))

### LOGPosterior

In [ ]:
y=data.Am$y
x=data.Am$x 
parametri <- as.data.frame(as.mcmc(total.chain.Am))



best.params <- c(0,0,0,0,0)
LogPostBest <- -Inf

for (j in 1:dim(total.chain.Am)[1]){

    J=parametri$J[j]
    A=parametri$A[j]
    mu=parametri$mu[j] 
    sigma=parametri$sigma[j]
    c.1=parametri$c.1[j]
    c.2=parametri$c.2[j]
    c.mu=parametri$c.mu[j]
    offset=parametri$offset[j]

  
    #LogLikelihood

    LogL <- 0

    for (i in 1:length(y)){

        point <- Heaviside(mu - J - x[i],a=0)*(c.1/(1 + exp(c.2*(x[i] - c.mu))) + offset +  A/(sigma)*exp(J*(2*x[i] - 2*mu + J)/(2*sigma*sigma))) +
                 Heaviside(x[i] - mu + J,a=0)*(c.1/(1 + exp(c.2*(x[i] - c.mu))) + offset + A/(sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)))


        p.y <- dpois(y[i], point) 

        LogL <- LogL + log(p.y)

    }
    

    #LogPrior
    p.A <- dunif(A,50000,500000)           
    p.mu <- dunif(mu,150,170)        
    p.sigma <- dgamma(sigma,60,7)          
    p.c.1 <- dunif(c.1,1000,4000)
    p.c.2 <- dunif(c.2,0,0.6) 
    p.c.mu <- dnorm(c.mu,mu,1/4)
    p.offset <- dunif(offset,500,2000) 
    p.J <- dunif(J,0,100)
 
    LogPrior <- log(p.A) + log(p.mu) + log(p.sigma) + log(p.c.1) + 
                log(p.sigma) + log(p.c.2) + log(p.c.mu) + log(p.offset) + log(p.J)


    #LogPost
    LogPost <- LogPrior + LogL

    #cat(paste("LogL:",LogL,"\n"))
    #cat(paste("LogPost:",LogPost,"\n"))

    
    
    #check for maximum of the posterior
    if(LogPost > LogPostBest){
        LogPostBest <- LogPost
        best.params <- c(A,J,  c.1, c.2, c.mu, mu, offset, sigma)
    }
    
}



In [ ]:
cat(best.params)

In [ ]:
mlu

In [ ]:
###confronto tra media, moda e originale
params=summary(chain.Am)$statistics[,1]
mlu <- mode.lower.upper.func(total.chain.Am)
#mlu



extracted <-  sapply(data.Am$x,piecewise,A=best.params[1],J=best.params[2],c.1=best.params[3],c.2=best.params[4],c.mu=best.params[5],mu=best.params[6],offset=best.params[7],sigma=best.params[8])
extracted.moda <-  sapply(data.Am$x, piecewise,A=mlu[1,1],J=mlu[2,1],c.1=mlu[3,1],c.2=mlu[4,1],c.mu=mlu[5,1],mu=mlu[6,1],offset=mlu[7,1],sigma=mlu[8,1])
extracted.mean <-  sapply(data.Am$x, piecewise,A=params[1],J=params[2],c.1=params[3],c.2=params[4],c.mu=params[5],mu=params[6],offset=params[7],sigma=params[8])

plot.func.total.1(data.Am$x,data.Am$y,extracted,extracted.moda,extracted.mean,title=("Americium E = 0.059 MeV"))

### Integration Routti-Prussin

In [ ]:
Routti.Prussin.peak.fix<- function(z)
    {
    return(Routti.Prussin.peak(z,A=best.params[1],J=best.params[2],mu=best.params[6],sigma=best.params[8]))
    }

In [ ]:
int2 <- integrate(Routti.Prussin.peak.fix,lower=(params[6]-1.55*sqrt(8*log(2))*params[8]), upper=(params[6]+1.55*sqrt(8*log(2))*params[8]))$value 

In [ ]:
int2

In [ ]:
params <- summary(chain.Am)$statistics[,1:2]
params

# CESIUM

In [ ]:
summary.table["Cs",]

In [ ]:
rangeCs = dataset[!(dataset$ADC_channel<=summary.table["Cs","range.gauss.min"]) & (dataset$ADC_channel<=summary.table["Cs","range.gauss.max"]),]
rangeCs <- data.frame(rangeCs["Counts"], rangeCs["ADC_channel"])
head(rangeCs)

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)
par(mfrow=c(1,2))
plot(rangeCs$ADC_channel,rangeCs$Counts, type = "p", col='red', main = 'Cs E = 0.66 MeV', xlab = 'ADC channel [#]', ylab = "Counts [#]",cex.main = 1.5)
plot(rangeCs$ADC_channel,log(rangeCs$Counts), type = "p", col='red', main = 'Cs E = 0.66 MeV', xlab = 'ADC channel [#]', ylab = "log(Counts) [#]",cex.main = 1.5)

## GAUSSIAN

In [ ]:
cat("model{
    #data likelihood
    for (i in 1:length(x)){

        points[i] <- N/sqrt(2*pi*sigma*sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)) +
                     c.1/(1+exp(c.2*(x[i]-c.mu))) + offset
                        
        y[i] ~ dpois(points[i])


    }
    #priors for the generative model

    N ~ dunif(100000,1500000)
    mu ~ dunif(1700,1800)
    sigma ~ dgamma(0.0085,0.001)
    c.1 ~ dunif(0,400)
    c.2 ~ dunif(0,0.4) 
    c.mu ~ dnorm(mu,1)
    offset ~ dunif(0,500) 

    

}", file="BUGS_model_Cs_S1.txt")
model.Cs.S1 <- "BUGS_model_Cs_S1.txt"


In [ ]:
init.val.Cs.S1.1 <- list(N =300000,mu = 1720,sigma =2, offset=300, c.mu = 1720, c.1=100, c.2 = 0.1,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Cs.S1.2 <- list(N =500000,mu = 1760,sigma =15, offset=400, c.mu = 1760, c.1=20, c.2 = 0.4, 
                    .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Cs.S1.3 <- list(N =600000,mu = 1740,sigma =10, offset=220, c.mu = 1740, c.1=200, c.2 = 0.003,
                    .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Cs.S1 <- list(init.val.Cs.S1.1 , init.val.Cs.S1.2 , init.val.Cs.S1.3)

data.Cs.S1 <- NULL
data.Cs.S1$x <- rangeCs$ADC_channel
data.Cs.S1$y <- rangeCs$Counts
data.Cs.S1$pi <- pi

In [ ]:
jm.Cs.S1 <- jags.model(model.Cs.S1, data.Cs.S1, init.val.Cs.S1,n.adapt=0,n.chains=3)
update(jm.Cs.S1, 1000)
chain.Cs.S1 <- coda.samples(jm.Cs.S1 , c("N","mu", "sigma","c.1","c.2","c.mu","offset"), n.iter=12000, thin=25) 

In [ ]:
#Gelman and Rubin's convergence diagnostic:ratio between variance between the chains and variance within a chain
gelman.diag(chain.Cs.S1)$psrf[,1]
cat(paste("Total chain R value:",gelman.diag(chain.Cs.S1)$ mpsrf))

In [ ]:
params=summary(chain.Cs.S1)$statistics[,1:2]
params

In [ ]:
total.chain.Cs.S1 <- mcmc(do.call(rbind, chain.Cs.S1),thin=thin(chain.Cs.S1)) #combines the three chains

In [ ]:
par(mfrow=c(4,2),mar=c(5,5.5,4,0.7))
options(repr.plot.width=15, repr.plot.height=30)

chain.Cs.S1.df <- as.data.frame ( as.mcmc( total.chain.Cs.S1 ) )

for (p in names(chain.Cs.S1.df)){
    plot.stats.2(total.chain.Cs.S1, p, leg=TRUE, digits=8)
}


VISUALIZATION OF THE CHAIN

In [ ]:
#VISUALIZATION OF THE CHAIN

options(repr.plot.width=10, repr.plot.height=15)
plot(chain.Cs.S1,cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF AUTOCORRELATION

In [ ]:
autocorr.plot(chain.Cs.S1.df,mfrow=c(3,3),cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF CORRELATION BETWEEN PARAMETERS

In [ ]:
plot.correlations(chain.Cs.S1.df,c(3,3))

### LOGPosterior

In [ ]:
y=data.Cs.S1$y
x=data.Cs.S1$x 
parametri <- as.data.frame(as.mcmc(total.chain.Cs.S1))



best.params <- c(0,0,0,0,0)
LogPostBest <- -Inf

for (j in 1:dim(total.chain.Cs.S1)[1]){

    
    N=parametri$N[j]
    mu=parametri$mu[j] 
    sigma=parametri$sigma[j]
    c.1=parametri$c.1[j]
    c.2=parametri$c.2[j]
    c.mu=parametri$c.mu[j]
    offset=parametri$offset[j]

  
    #LogLikelihood

    LogL <- 0

    for (i in 1:length(y)){

        point <- N/sqrt(2*pi*sigma*sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)) + #+  #normal generative model
                                c.1/(1+exp(c.2*(x[i]-c.mu))) + offset  #line generative model

        p.y <- dpois(y[i], point) 

        LogL <- LogL + log(p.y)

    }
    

    #LogPrior
    p.N <- dunif(N,100000,1500000)           
    p.mu <- dunif(mu,1700,1800)        
    p.sigma <- dgamma(sigma,60,7)          
    p.c.1 <- dunif(c.1,0,400)
    p.c.2 <- dunif(c.2,0,0.6) 
    p.c.mu <- dnorm(c.mu,mu,1)
    p.offset <- dunif(offset,0,500) # E' il valore del plateau a dx
    
    LogPrior <- log(p.N) + log(p.mu) + log(p.sigma) + log(p.c.1) + 
                log(p.sigma) + log(p.c.2) + log(p.c.mu) + log(p.offset)


    #LogPost
    LogPost <- LogPrior + LogL

    #cat(paste("LogL:",LogL,"\n"))
    #cat(paste("LogPost:",LogPost,"\n"))

    
    
    #check for maximum of the posterior
    if(LogPost > LogPostBest){
        LogPostBest <- LogPost
        best.params <- c(N, c.1, c.2, c.mu, mu, offset, sigma)
    }
    
}
best.params[2] <- best.params[2]


In [ ]:
cat(best.params)

In [ ]:
###confronto tra media, moda e originale
params=summary(chain.Cs.S1)$statistics[,1]
mlu <- mode.lower.upper.func(total.chain.Cs.S1)
mlu



extracted <-  sapply(data.Cs.S1$x,Gauss.Sig.func, N=best.params[1],c.1=best.params[2],c.2=best.params[3],c.mu=best.params[4],mu=best.params[5],offset=best.params[6],sigma=best.params[7])
extracted.moda <-  sapply(data.Cs.S1$x, Gauss.Sig.func,N=mlu[1,1],c.1=mlu[2,1],c.2=mlu[3,1],c.mu=mlu[4,1],mu=mlu[5,1],offset=mlu[6,1],sigma=mlu[7,1])
extracted.mean <-  sapply(data.Cs.S1$x, Gauss.Sig.func,N=params[1],c.1=params[2],c.2=params[3],c.mu=params[4],mu=params[5],offset=params[6],sigma=params[7])


plot.func.total.1(data.Cs.S1$x,data.Cs.S1$y,extracted,extracted.moda,extracted.mean,title=("Cesium (Gaussian + Sigmoid) E = 0.66 MeV"))

## Routti and Prussin Peak-Shape Function (1969) [1] 

In [ ]:
cat("model{
    #data likelihood
    for (i in 1:length(x)){

                        
    points[i] <- step(mu - J - x[i])*(c.1/(1 + exp(c.2*(x[i] - c.mu))) + offset +  A/(sigma)*exp(J*(2*x[i] - 2*mu + J)/(2*sigma*sigma))) +
                 step(x[i] - mu + J)*(c.1/(1 + exp(c.2*(x[i] - c.mu))) + offset + A/(sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)))

    y[i] ~ dpois(points[i])

    }
    #priors for the generative model
    A ~ dunif(50000,1000000)
    mu ~ dunif(1700,1800)
    sigma ~ dgamma(0.0085,0.001)
    c.1 ~ dunif(0,400)
    c.2 ~ dunif(0,0.5)  
    c.mu ~ dnorm(mu,1/4)
    J ~ dunif(0,100)
    offset ~ dunif(220,500) 

}", file="Bmodel.cesium.txt")

model.Cs <- "Bmodel.cesium.txt"


In [ ]:
#MODEL SET UP: INITIAL VALUES AND DATA PREPARATION

#we make an estimate of the true values looking at the plot
#TRUE values are unkown


init.val.Cs.1 <- list(A =800000,mu = 1720,sigma =2, offset=300, c.mu = 1720, c.1=100, c.2 = 0.1, J=5,
                     .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Cs.2 <- list(A =250000,mu = 1800,sigma =15, offset=400, c.mu = 1800, c.1=20, c.2 = 0.5, J=1,
                    .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Cs.3 <- list(A =500000,mu = 1700,sigma =10, offset=220, c.mu = 1700, c.1=200, c.2 = 0.01, J=90,
                    .RNG.seed  = 1996,.RNG.name = "base::Mersenne-Twister")
init.val.Cs <- list(init.val.Cs.1 , init.val.Cs.2 , init.val.Cs.3)

data.Cs <- NULL
data.Cs$x <- rangeCs$ADC_channel
data.Cs$y <- rangeCs$Counts

In [ ]:
#reading the model
jmCs <- jags.model(model.Cs, data.Cs, inits=init.val.Cs, n.adapt=0, n.chains=3)
update(jmCs, 1000)
chain.Cs <- coda.samples(jmCs , c("A","mu", "sigma","c.1","c.2","c.mu","offset","J"),n.iter=10000, thin=20) 

In [ ]:
#Gelman and Rubin's convergence diagnostic:ratio between variance between the chains and variance within a chain
gelman.diag(chain.Cs)$psrf[,1]
cat(paste("Total chain R value:",gelman.diag(chain.Cs)$ mpsrf))

In [ ]:
params=summary(chain.Cs)$statistics[,1:2]
params

In [ ]:
total.chain.Cs <- mcmc(do.call(rbind, chain.Cs),thin=thin(chain.Cs)) #combines the three chains

In [ ]:
par(mfrow=c(4,2),mar=c(5,5.5,4,0.7))
options(repr.plot.width=15, repr.plot.height=30)

chain.Cs.df <- as.data.frame ( as.mcmc( total.chain.Cs ) )

for (p in names(chain.Cs.df)){
    plot.stats.2(total.chain.Cs, p, leg=TRUE, digits=8)
}


VISUALIZATION OF THE CHAIN



In [ ]:
options(repr.plot.width=10, repr.plot.height=15)
plot(chain.Cs,cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF AUTOCORRELATION

In [ ]:
autocorr.plot(chain.Cs.df,cex.lab=1.3, cex.axis=1.3, cex.main=1.3, cex.sub=1.3)

VISUALIZATION OF CORRELATION BETWEEN VARIABLES

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
plot.correlations(chain.Cs.df, c(3,4))

###  LOGPosterior

In [ ]:
y=data.Cs$y
x=data.Cs$x 
parametri <- as.data.frame(as.mcmc(total.chain.Cs))



best.params <- c(0,0,0,0,0)
LogPostBest <- -Inf

for (j in 1:dim(total.chain.Cs)[1]){

    J=parametri$J[j]
    A=parametri$A[j]
    mu=parametri$mu[j] 
    sigma=parametri$sigma[j]
    c.1=parametri$c.1[j]
    c.2=parametri$c.2[j]
    c.mu=parametri$c.mu[j]
    offset=parametri$offset[j]

  
    #LogLikelihood

    LogL <- 0

    for (i in 1:length(y)){

        point <- Heaviside(mu - J - x[i],a=0)*(c.1/(1 + exp(c.2*(x[i] - c.mu))) + offset +  A/(sigma)*exp(J*(2*x[i] - 2*mu + J)/(2*sigma*sigma))) +
                 Heaviside(x[i] - mu + J,a=0)*(c.1/(1 + exp(c.2*(x[i] - c.mu))) + offset + A/(sigma)*exp((-(x[i]-mu)*(x[i]-mu))/(2*sigma*sigma)))


        p.y <- dpois(y[i], point) 

        LogL <- LogL + log(p.y)

    }
    

    #LogPrior
    p.A <- dunif(A,100000,1500000)           
    p.mu <- dunif(mu,1700,1800)        
    p.sigma <- dgamma(sigma,0.0085,0.001)          
    p.c.1 <- dunif(c.1,0,400)
    p.c.2 <- dunif(c.2,0,200) 
    p.c.mu ~ dunif(c.mu,mu-5,mu+5)
    p.offset <- dunif(offset,220,500) 
    p.J <- dunif(J,0,100)
 
    LogPrior <- log(p.A) + log(p.mu) + log(p.sigma) + log(p.c.1) + 
                log(p.sigma) + log(p.c.2) + log(p.c.mu) + log(p.offset) + log(p.J)


    #LogPost
    LogPost <- LogPrior + LogL

    #cat(paste("LogL:",LogL,"\n"))
    #cat(paste("LogPost:",LogPost,"\n"))

    
    
    #check for maximum of the posterior
    if(LogPost > LogPostBest){
        LogPostBest <- LogPost
        best.params <- c(A, J, c.1, c.2, c.mu, mu, offset, sigma)
    }
    
}



In [ ]:
cat(best.params)

In [ ]:
###confronto tra media, moda e originale
params=summary(chain.Cs)$statistics[,1]
mlu <- mode.lower.upper.func(total.chain.Cs)
mlu



extracted <-  sapply(data.Cs$x,piecewise,A=best.params[1],J=best.params[2],c.1=best.params[3],c.2=best.params[4],c.mu=best.params[5],mu=best.params[6],offset=best.params[7],sigma=best.params[8])
extracted.moda <-  sapply(data.Cs$x, piecewise,A=mlu[1,1],J=mlu[2,1],c.1=mlu[3,1],c.2=mlu[4,1],c.mu=mlu[5,1],mu=mlu[6,1],offset=mlu[7,1],sigma=mlu[8,1])
extracted.mean <-  sapply(data.Cs$x, piecewise,A=params[1],J=params[2],c.1=params[3],c.2=params[4],c.mu=params[5],mu=params[6],offset=params[7],sigma=params[8])


plot.func.total.1(data.Cs$x,data.Cs$y,extracted,extracted.moda,extracted.mean,title=("Cesium E = 0.66 MeV"))

### Integration Routti-Prussin

In [ ]:
Routti.Prussin.peak.fix<- function(z)
    {
    return(Routti.Prussin.peak(z,A=best.params[1],J=best.params[2],mu=best.params[6],sigma=best.params[8]))
    }

In [ ]:
int2 <- integrate(Routti.Prussin.peak.fix,lower=(params[6]-1.55*sqrt(8*log(2))*params[8]), upper=(params[6]+1.55*sqrt(8*log(2))*params[8]))$value 
int2

# FINAL RESULTS

The final goal of this analysis was to evaluate the number of counts under each of the considered peaks. In the following table we gather possible estimators for N.

The **mode** is the best estimator we can obtain from the marginalized distribution; we also compute the **mean**  of the marginalized distribution since it is very common to consider it in literature.

If we want to get the value of N that belongs to the set of parameters with **highest probability** for our model, we have to compute the **posterior distribution** , look for the set of sampled parameters that maximazes it and take the value of N from there.

| Nucleus                   	        | Model               	| $N_{MAP}$ 	| $N_{mean}$ 	| $\sigma_{N_{mean}}$ 	| $N_{mode}$ 	| Lower limit 	| Upper limit 	|
|---------------------------	|---------------------	|-----------	|------------	|----------	|----------	|-------------	|-------------	|
| $^{241\!\,}Am$            	  | Gaussian + Sigmoid  	|         168633  	|   169130	         	|   693       	|         169250	 	|        168446     	|      169801       	|
| $^{137\!\,}Cs$             	  | Gaussian + Sigmoid 	|     503681      	|   503635	         	|    731      	|          	    503750	        	|       	502869      	| 504322
| $^{60\!\,}Co_{1} $ 	      | Gaussian + Exp      	|   179436        	|  179468          	| 426         	|  179500      	|    179031        	 |        179875      	|
| $^{60\!\,}Co_{2}  $   	  | Gaussian + Exp      	|   		163803        	|     162879     	| 406          	|  162900	       	|   162437          	|        163247     	|
| Double absorption       	| Gaussian + Exp      	|  11468         	|    11464	      	|     108     	| 11475         	|       11373      	|      11589       

| Nucleus                   	| N via Routti-Proussin Integration  | 
|---------------------------	|---------------------	
| $^{241\!\,}Am$            	|          175067                    |
| $^{137\!\,}Cs$             	|          506239              |

Since it is the only quantity we have a true estimate of, we also show the values of energy corresponding to the $\mu$ of the Gaussians, i.e. the energy value at which we get the maximum value of the peak, and compare it with its well known theoretical value [2].

Looking at the marginalized distributions of $\mu$ we can recognize an approximately Gaussian behaviour; for this reason we used the $z$-test to check for compatibility with the thoeretical value. A more rigorous approach would be first check for compatibility of the marginalized distributions with the Gaussian one using for example a fit and an hypothesis test. 

| Nucleus                     | Model            | $E_{The}$ [KeV] |$\mu$  [KeV]|$\sigma_{\mu}$ | $z_{test}$| 
|---------------------------  |------------------|-----------------|-----------	|------------	|----------	|
| $^{241\!\,}Am$              | Gaussian + Sigmoid	     |        59.541       |  58        |1       |1.5|
| $^{137\!\,}Cs$              | Gaussian + Sigmoid 	     |   	  661.659      |  661.6       |0.8       |0.07|
| $^{60\!\,}Co_{1} $ 	      | Gaussian + Exp   |        1173.240 	   |  1174.0      |0.7      |1.1 |
| $^{60\!\,}Co_{2}  $   	  | Gaussian + Exp   |   	  1332.508     |  1333.6      |0.7       |1.6|
| Double absorption       	  | Gaussian + Exp   |        2505.748 	   |  2505  	|1       |0.7|

# REFERENCE

[1] [R.G. Helmer, M.A. Lee, Analytical functions for fitting peaks from Ge semiconductor detectors](https://www.sciencedirect.com/science/article/abs/pii/0029554X80908307)

[2] [Laboratoire national Henri Becquerel, tables of eveluated data on radioactive nuclides](http://www.nucleide.org/DDEP_WG/DDEPdata.htm) 